In [57]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [58]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [59]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) 
spark

In [61]:
!wget https://raw.githubusercontent.com/amit2001cse/Dataset-1/main/Mall_Customers.csv
df = spark.read.csv('/content/Mall_Customers.csv', header=True)
df.show(5)

--2022-09-13 07:49:08--  https://raw.githubusercontent.com/amit2001cse/Dataset-1/main/Mall_Customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3979 (3.9K) [text/plain]
Saving to: ‘Mall_Customers.csv.9’

Mall_Customers.csv. 100%[===================>]   3.89K  --.-KB/s    in 0s      

2022-09-13 07:49:08 (51.3 MB/s) - ‘Mall_Customers.csv.9’ saved [3979/3979]

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
| 

In [62]:
from pyspark.sql.types import *
df.columns
# Creating a list of the schema in the format column_name, data_type
labels = [
    ('CustomerId',IntegerType()),
    ('Gender',StringType()),
    ('Age',IntegerType()),
    ('Annual Income (k$)',IntegerType()),
    ('Spending Score (1-100)',IntegerType())
]
# Creating the schema that will be passed when reading the csv
schema = StructType([StructField (x[0], x[1], True) for x in labels])
schema

StructType(List(StructField(CustomerId,IntegerType,true),StructField(Gender,StringType,true),StructField(Age,IntegerType,true),StructField(Annual Income (k$),IntegerType,true),StructField(Spending Score (1-100),IntegerType,true)))

In [63]:
df = spark.read.csv('Mall_Customers.csv', header=True,  schema = schema)
df.show(5)

+----------+------+---+------------------+----------------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+
only showing top 5 rows



In [64]:
print(df.CustomerId)
print("*"*20)
df.select(df.CustomerId).show(truncate=False)

Column<'CustomerId'>
********************
+----------+
|CustomerId|
+----------+
|1         |
|2         |
|3         |
|4         |
|5         |
|6         |
|7         |
|8         |
|9         |
|10        |
|11        |
|12        |
|13        |
|14        |
|15        |
|16        |
|17        |
|18        |
|19        |
|20        |
+----------+
only showing top 20 rows



In [65]:
print(df['CustomerId'])
print("*"*20)
df.select(df['CustomerId']).show(truncate=False)

# 3rd method
# Column name is case insensitive here
from pyspark.sql.functions import col
df.select(col('CustomerId')).show(truncate=False)

Column<'CustomerId'>
********************
+----------+
|CustomerId|
+----------+
|1         |
|2         |
|3         |
|4         |
|5         |
|6         |
|7         |
|8         |
|9         |
|10        |
|11        |
|12        |
|13        |
|14        |
|15        |
|16        |
|17        |
|18        |
|19        |
|20        |
+----------+
only showing top 20 rows

+----------+
|CustomerId|
+----------+
|1         |
|2         |
|3         |
|4         |
|5         |
|6         |
|7         |
|8         |
|9         |
|10        |
|11        |
|12        |
|13        |
|14        |
|15        |
|16        |
|17        |
|18        |
|19        |
|20        |
+----------+
only showing top 20 rows



In [66]:
print(df.CustomerId, df.Gender)
print("*"*40)
df.select(df.CustomerId, df.Gender).show(truncate=False)
#Anoter Way
print(df['CustomerId'],df['Gender'])
print("*"*40)
df.select(df['CustomerId'],df['Gender']).show(truncate=False)
#Another Way
from pyspark.sql.functions import col
df.select(col('CustomerId'),col('Gender')).show(truncate=False)

Column<'CustomerId'> Column<'Gender'>
****************************************
+----------+------+
|CustomerId|Gender|
+----------+------+
|1         |Male  |
|2         |Male  |
|3         |Female|
|4         |Female|
|5         |Female|
|6         |Female|
|7         |Female|
|8         |Female|
|9         |Male  |
|10        |Female|
|11        |Male  |
|12        |Female|
|13        |Female|
|14        |Female|
|15        |Male  |
|16        |Male  |
|17        |Female|
|18        |Male  |
|19        |Male  |
|20        |Female|
+----------+------+
only showing top 20 rows

Column<'CustomerId'> Column<'Gender'>
****************************************
+----------+------+
|CustomerId|Gender|
+----------+------+
|1         |Male  |
|2         |Male  |
|3         |Female|
|4         |Female|
|5         |Female|
|6         |Female|
|7         |Female|
|8         |Female|
|9         |Male  |
|10        |Female|
|11        |Male  |
|12        |Female|
|13        |Female|
|14        |Fema

In [67]:
from pyspark.sql.functions import lit
df = df.withColumn('first_column',lit(1))

df.show(5,truncate=False)

+----------+------+---+------------------+----------------------+------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|first_column|
+----------+------+---+------------------+----------------------+------------+
|1         |Male  |19 |15                |39                    |1           |
|2         |Male  |21 |15                |81                    |1           |
|3         |Female|20 |16                |6                     |1           |
|4         |Female|23 |16                |77                    |1           |
|5         |Female|31 |17                |40                    |1           |
+----------+------+---+------------------+----------------------+------------+
only showing top 5 rows



In [68]:
df = df.withColumn('second_column', lit(2)) \
      .withColumn('third_column', lit('Third Column'))
df.show(5,truncate=False)

+----------+------+---+------------------+----------------------+------------+-------------+------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|first_column|second_column|third_column|
+----------+------+---+------------------+----------------------+------------+-------------+------------+
|1         |Male  |19 |15                |39                    |1           |2            |Third Column|
|2         |Male  |21 |15                |81                    |1           |2            |Third Column|
|3         |Female|20 |16                |6                     |1           |2            |Third Column|
|4         |Female|23 |16                |77                    |1           |2            |Third Column|
|5         |Female|31 |17                |40                    |1           |2            |Third Column|
+----------+------+---+------------------+----------------------+------------+-------------+------------+
only showing top 5 rows



In [69]:
from pyspark.sql.functions import concat
df = df.withColumn('CustomerId_Age', concat(col("CustomerId"), lit(" "), col("Age")))

df.show(5,truncate=False)

+----------+------+---+------------------+----------------------+------------+-------------+------------+--------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|first_column|second_column|third_column|CustomerId_Age|
+----------+------+---+------------------+----------------------+------------+-------------+------------+--------------+
|1         |Male  |19 |15                |39                    |1           |2            |Third Column|1 19          |
|2         |Male  |21 |15                |81                    |1           |2            |Third Column|2 21          |
|3         |Female|20 |16                |6                     |1           |2            |Third Column|3 20          |
|4         |Female|23 |16                |77                    |1           |2            |Third Column|4 23          |
|5         |Female|31 |17                |40                    |1           |2            |Third Column|5 31          |
+----------+------+---+---------

In [70]:
#Renaming a column in PySpark
df = df.withColumnRenamed('first_column', 'new_column_one') \
      .withColumnRenamed('second_column', 'new_column_two') \
      .withColumnRenamed('third_column', 'new_column_three')
df.show(truncate=False)

+----------+------+---+------------------+----------------------+--------------+--------------+----------------+--------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|new_column_one|new_column_two|new_column_three|CustomerId_Age|
+----------+------+---+------------------+----------------------+--------------+--------------+----------------+--------------+
|1         |Male  |19 |15                |39                    |1             |2             |Third Column    |1 19          |
|2         |Male  |21 |15                |81                    |1             |2             |Third Column    |2 21          |
|3         |Female|20 |16                |6                     |1             |2             |Third Column    |3 20          |
|4         |Female|23 |16                |77                    |1             |2             |Third Column    |4 23          |
|5         |Female|31 |17                |40                    |1             |2             |Third Col

In [71]:
#Remove columns in PySpark
df = df.drop('new_column_one')
df.show(5,truncate=False)

#Remove multiple columnss in one go
df = df.drop('new_column_two') \
      .drop('new_column_three')
df.show(5,truncate=False)


+----------+------+---+------------------+----------------------+--------------+----------------+--------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|new_column_two|new_column_three|CustomerId_Age|
+----------+------+---+------------------+----------------------+--------------+----------------+--------------+
|1         |Male  |19 |15                |39                    |2             |Third Column    |1 19          |
|2         |Male  |21 |15                |81                    |2             |Third Column    |2 21          |
|3         |Female|20 |16                |6                     |2             |Third Column    |3 20          |
|4         |Female|23 |16                |77                    |2             |Third Column    |4 23          |
|5         |Female|31 |17                |40                    |2             |Third Column    |5 31          |
+----------+------+---+------------------+----------------------+--------------+----------------

In [72]:
# Filtering rows in PySpark
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
male_filtered_count = df.filter(col('Gender')=='Male').count()
print("Male FILTERED RECORD COUNT: " + str(male_filtered_count))
df.filter(col('Gender')=='Male').show(truncate=False)

TOTAL RECORD COUNT: 200
Male FILTERED RECORD COUNT: 88
+----------+------+---+------------------+----------------------+--------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|CustomerId_Age|
+----------+------+---+------------------+----------------------+--------------+
|1         |Male  |19 |15                |39                    |1 19          |
|2         |Male  |21 |15                |81                    |2 21          |
|9         |Male  |64 |19                |3                     |9 64          |
|11        |Male  |67 |19                |14                    |11 67         |
|15        |Male  |37 |20                |13                    |15 37         |
|16        |Male  |22 |20                |79                    |16 22         |
|18        |Male  |20 |21                |66                    |18 20         |
|19        |Male  |52 |23                |29                    |19 52         |
|21        |Male  |35 |24                |35          

In [73]:
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
female_filtered_count = df.filter((col('Gender')=='Female') &
                                 (col('Age')==21)).count() # Two conditions added here
print("Female FILTERED RECORD COUNT: " + str(female_filtered_count))
df.filter(col('Gender')=='Female').show(truncate=False)

TOTAL RECORD COUNT: 200
Female FILTERED RECORD COUNT: 4
+----------+------+---+------------------+----------------------+--------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|CustomerId_Age|
+----------+------+---+------------------+----------------------+--------------+
|3         |Female|20 |16                |6                     |3 20          |
|4         |Female|23 |16                |77                    |4 23          |
|5         |Female|31 |17                |40                    |5 31          |
|6         |Female|22 |17                |76                    |6 22          |
|7         |Female|35 |18                |6                     |7 35          |
|8         |Female|23 |18                |94                    |8 23          |
|10        |Female|30 |19                |72                    |10 30         |
|12        |Female|35 |19                |99                    |12 35         |
|13        |Female|58 |20                |15         

In [74]:
df.select('Gender').distinct().show()

df.select('Gender','Annual Income (k$)').distinct().show()

+------+
|Gender|
+------+
|Female|
|  Male|
+------+

+------+------------------+
|Gender|Annual Income (k$)|
+------+------------------+
|Female|                47|
|Female|                20|
|Female|                99|
|Female|                65|
|Female|               101|
|Female|                74|
|  Male|                73|
|Female|                37|
|  Male|                15|
|  Male|                23|
|  Male|                71|
|  Male|                21|
|  Male|                69|
|Female|                38|
|Female|               120|
|Female|                98|
|  Male|                93|
|  Male|                33|
|Female|                76|
|Female|                97|
+------+------------------+
only showing top 20 rows



In [75]:
# By default the data will be sorted in ascending order
df.orderBy('Age').show(truncate=False)

# To change the sorting order, you can use the ascending parameter
df.orderBy('Age', ascending=False).show(truncate=False)

# Using groupBy aand orderBy together
df.groupBy("Gender").count().orderBy('count', ascending=False).show(10)

+----------+------+---+------------------+----------------------+--------------+
|CustomerId|Gender|Age|Annual Income (k$)|Spending Score (1-100)|CustomerId_Age|
+----------+------+---+------------------+----------------------+--------------+
|66        |Male  |18 |48                |59                    |66 18         |
|92        |Male  |18 |59                |41                    |92 18         |
|115       |Female|18 |65                |48                    |115 18        |
|34        |Male  |18 |33                |92                    |34 18         |
|1         |Male  |19 |15                |39                    |1 19          |
|69        |Male  |19 |48                |59                    |69 19         |
|139       |Male  |19 |74                |10                    |139 19        |
|163       |Male  |19 |81                |5                     |163 19        |
|62        |Male  |19 |46                |55                    |62 19         |
|114       |Male  |19 |64   